# Overview and Goals:

This will be an early iteration to see if I can get any improvement of classification schemes. For inputs, it will be important to consider the ratio of training stars to training galaxies. I'm not sure exactly what I want to do yet, but it will be a factor. Include quality and isochrone cuts, just like the real data would have. After that, inputs will consist of the psf (if I can obtain some numbers for this), different bands and different types of magnitude, measured size, original class given, essentially any object property that I also have access to in the GOLD catalog, probably flux, and every survey property from the measured detection pixel. Hold out a testing dataset, and when testing MAKE SURE to look at stars and galaxies separately. A high accuracy because everything is classified as a galaxy is unacceptable. Compare accuracy results to pre correction results. Consider doing an RNN, this should make the classification scheme unable to perform worse since it is already given the original class. For outputs, do a binary system. We could do floating points, but I think categorical cross entropy might work better for what I want to get out of this. For testing, also look at rates within multiple different color magnitude bins.

In [1]:
import numpy as np
import tensorflow as tf
# import keras
# from keras import layers
import StellarConfig as strConfig
from matplotlib.path import Path
import BalrogTestConfig as balrConfig
import fitsio
import healpy as hp
import matplotlib.pyplot as plt

In [2]:
baseDirectory = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/NeuralNetworkClassification/'

In [3]:
# This includes relevant object properties AND POSITIONS TO GET PIXELS
starDataFile = baseDirectory + 'Star_Balrog_Data.fits'
galaDataFile = baseDirectory + 'Galaxy_Balrog_Data.fits'

### 4096 Valid Pix (Only Run Once to Generate)

In [4]:
validPixFile = baseDirectory + 'Valid_Pixels_4096.fits'

In [5]:
# origCondFiles = balrConfig.origCondFiles
# stelFile = balrConfig.stelFile

# validPix = np.full(12*(4096**2), True, dtype = bool)
    
# for file in origCondFiles:
#     condData = fitsio.read(file) # This reads in the data.
#     condSigExt = np.full(12*(4096**2), -1.6375e+30) # Gives a default value.
#     condSigExt[condData['PIXEL']] = condData['SIGNAL'] # Changes all valid pixels to their corresponding signals.
#     validPix[np.where(condSigExt < -100)] = False # An invalid survey property marks this pixel as false.

# stelDensExt = fitsio.read(stelFile)['I'].flatten() # Loads in the stellar density.
# validPix[np.where(stelDensExt < -100)] = False # Any non valid pixel has its value changed to False.

# validPix = np.where(validPix)[0]

In [6]:
# from astropy.table import Table

# my_table = Table()
# my_table['PIXELS'] = validPix
# my_table.write(validPixFile, overwrite = True)

In [7]:
validPix = fitsio.read(validPixFile)['PIXELS']

In [8]:
def trainValTest(length, trainPer, valPer):
    indices = np.arange(length)
    np.random.shuffle(indices)
    trainIndices = indices[0:int(trainPer*length)]
    valIndices = indices[int(trainPer*length):int((trainPer + valPer)*length)]
    testIndices = indices[int((trainPer + valPer)*length):]
    return trainIndices, valIndices, testIndices

### Parameters to Use (Appeared in Both Star and Galaxy Catalogs)

meas_nimage_tot (4 bands)

meas_flags OBSOLETE DO NOT USE (all the same value on my data).

meas_box_size

meas_obj_flags OBSOLETE DO NOT USE (all the same value on my data).

meas_psf_flags (4 bands) OBSOLETE DO NOT USE (all the same value on my data).

meas_psf_flux (4 bands)

meas_psf_flux_err (4 bands)

meas_psf_mag (4 bands)

meas_psf_flux_s2n (4 bands)

meas_nimage_use (4 bands)

meas_mask_frac

meas_gapflux (4 bands)

meas_gapflags (4 bands) OBSOLETE DO NOT USE (all the same value on my data).

meas_cm_flags OBSOLETE DO NOT USE (all the same value on my data).

meas_cm_T

meas_cm_T_err

meas_cm_T_s2n

meas_cm_flux (4 bands)

meas_cm_flux_s2n (4 bands)

meas_cm_mag (4 bands)

meas_cm_logsb (4 bands)

meas_cm_g (2 components)

meas_cm_weight OBSOLETE DO NOT USE (all the same value on my data).

meas_cm_chi2per

meas_cm_dof

meas_cm_fracdev

meas_cm_fracdev_err

meas_cm_TdByTe

meas_EXTENDED_CLASS_SOF

57 total object properties

All 92 survey properties

In [9]:
parameterCount = 149

In [10]:
objectProps = ['meas_nimage_tot', 'meas_box_size',
               'meas_psf_flux', 'meas_psf_flux_err', 'meas_psf_mag',
               'meas_psf_flux_s2n', 'meas_nimage_use', 'meas_mask_frac', 'meas_gapflux',
               'meas_cm_T', 'meas_cm_T_err',
               'meas_cm_T_s2n', 'meas_cm_flux', 'meas_cm_flux_s2n', 'meas_cm_mag',
               'meas_cm_logsb', 'meas_cm_g', 'meas_cm_chi2per',
               'meas_cm_dof', 'meas_cm_fracdev', 'meas_cm_fracdev_err', 'meas_cm_TdByTe',
               'meas_EXTENDED_CLASS_SOF']

# Unnecessary for network, but needed for quality cuts
positionProps = ['meas_ra', 'meas_dec']
balrogID = ['bal_id']
cutFlags = ['flags_foreground', 'flags_badregions', 'flags_footprint', 'match_flag_1.5_asec']

# For galaxies only, to match to deep fields
trueID = ['true_id']

### Cuts to Make:

Color cut: cut to the phoenix isochrone. This model is not intended to be super general. Rather, it would be trained individually for each stellar stream. (Use magnitude corresponding to original class of star or galaxy).

Magnitude cut: perform a maximum r magnitude cut at 24.5, as in current algorithms. (Use magnitude corresponding to original class of star or galaxy).

Valid pix cut: self explanatory, cut to where the objects are in valid 4096 pixels.

Quality cut: perform the quality cuts performed at other points in this algorithm (foreground, badregions, footprint, arcseconds, and class being above or equal to 0).

Galaxy cut: obviously for balrog galaxies make sure they're modeled off of deep field galaxies.

In [11]:
res = 4096
classCutoff = 1.5
rMagCut = 24.5
path = strConfig.path
mu = strConfig.mu

In [12]:
# matBalrStarFile = strConfig.matBalrStarFile
# detBalrStarFile = strConfig.detBalrStarFile

# matBalrGalaFile = strConfig.matBalrFile
# detBalrGalaFile = strConfig.detBalrFile

In [13]:
# matStarData = fitsio.read(matBalrStarFile, columns = objectProps)
# starCutFlags = fitsio.read(detBalrStarFile, columns = cutFlags)
# starPositions = fitsio.read(matBalrStarFile, columns = positionProps)
# matStarID = fitsio.read(matBalrStarFile, columns = balrogID)
# detStarID = fitsio.read(detBalrStarFile, columns = balrogID)

In [14]:
# matGalaData = fitsio.read(matBalrGalaFile, columns = objectProps)
# galaCutFlags = fitsio.read(detBalrGalaFile, columns = cutFlags)
# galaPositions = fitsio.read(matBalrGalaFile, columns = positionProps)
# matGalaID = fitsio.read(matBalrGalaFile, columns = balrogID)
# detGalaID = fitsio.read(detBalrGalaFile, columns = balrogID)
# galaTrueID = fitsio.read(matBalrGalaFile, columns = trueID)

## Star Data

In [15]:
# STAR_GMAG = matStarData['meas_psf_mag'][:,0]
# STAR_RMAG = matStarData['meas_psf_mag'][:,1]

# STAR_CLASS = matStarData['meas_EXTENDED_CLASS_SOF']

# STAR_GMAG_CM = matStarData['meas_cm_mag'][:,0]
# STAR_RMAG_CM = matStarData['meas_cm_mag'][:,1]

# STAR_GMAG[np.where(STAR_CLASS >= classCutoff)[0]] = STAR_GMAG_CM[np.where(STAR_CLASS >= classCutoff)[0]]
# STAR_RMAG[np.where(STAR_CLASS >= classCutoff)[0]] = STAR_RMAG_CM[np.where(STAR_CLASS >= classCutoff)[0]]

# STAR_RA = starPositions['meas_ra']
# STAR_DEC = starPositions['meas_dec']
# STAR_PIX = hp.ang2pix(res, STAR_RA, STAR_DEC, nest = True, lonlat = True)

# STAR_MAT_ID = matStarID['bal_id']
# STAR_MAT_SORT = STAR_MAT_ID.argsort()

# STAR_FLAG_ID = detStarID['bal_id']

# FOREGROUND = starCutFlags['flags_foreground']
# BADREGIONS = starCutFlags['flags_badregions']
# FOOTPRINT = starCutFlags['flags_footprint']
# ARCSECONDS = starCutFlags['match_flag_1.5_asec']

# FLAG_SORT = STAR_FLAG_ID.argsort()
# STAR_FLAG_ID = STAR_FLAG_ID[FLAG_SORT]
# FOREGROUND = FOREGROUND[FLAG_SORT]
# BADREGIONS = BADREGIONS[FLAG_SORT]
# FOOTPRINT = FOOTPRINT[FLAG_SORT]
# ARCSECONDS = ARCSECONDS[FLAG_SORT]
    
# FLAG_CROP = np.isin(STAR_FLAG_ID, STAR_MAT_ID)

# FOREGROUND = FOREGROUND[FLAG_CROP]
# BADREGIONS = BADREGIONS[FLAG_CROP]
# FOOTPRINT = FOOTPRINT[FLAG_CROP]
# ARCSECONDS = ARCSECONDS[FLAG_CROP]

# FOREGROUND_PAR = np.zeros_like(FOREGROUND)
# BADREGIONS_PAR = np.zeros_like(FOREGROUND)
# FOOTPRINT_PAR = np.zeros_like(FOREGROUND)
# ARCSECONDS_PAR = np.zeros_like(FOREGROUND)

# FOREGROUND_PAR[STAR_MAT_SORT] = FOREGROUND
# BADREGIONS_PAR[STAR_MAT_SORT] = BADREGIONS
# FOOTPRINT_PAR[STAR_MAT_SORT] = FOOTPRINT
# ARCSECONDS_PAR[STAR_MAT_SORT] = ARCSECONDS

# STAR_MG = STAR_GMAG - mu
# STAR_GR = STAR_GMAG - STAR_RMAG

# filterSelection=Path.contains_points(path,np.vstack((STAR_GR,STAR_MG)).T)

# PIX_CUT = np.isin(STAR_PIX, validPix)

# starCrop = np.where((FOREGROUND_PAR == 0) &
#                     (BADREGIONS_PAR < 2) & 
#                     (FOOTPRINT_PAR == 1) &
#                     (ARCSECONDS < 2) &
#                     (STAR_RMAG <= rMagCut) &
#                     (STAR_CLASS >= 0) &
#                     (STAR_CLASS <= 3) & 
#                     (PIX_CUT) & 
#                     (filterSelection))[0]

In [16]:
# from astropy.table import Table

# star_table = Table()
# for prop in objectProps:
#     star_table[prop] = matStarData[prop][starCrop]
# for posProp in positionProps:
#     star_table[posProp] = starPositions[posProp][starCrop]
# star_table.write(starDataFile, overwrite = True)

In [17]:
starData = fitsio.read(starDataFile)

## Galaxy Data

In [18]:
# deepFiles = strConfig.deepFiles

# deepCols  = ['KNN_CLASS', 'ID']
# deepID = []
# deepClass= []

# for deepFile in deepFiles:
#     deepData = fitsio.read(deepFile, columns = deepCols)
#     deepID.extend(deepData['ID'])
#     deepClass.extend(deepData['KNN_CLASS'])

# deepID = np.array(deepID)
# deepClass = np.array(deepClass)

# # This serves to make it easier to check the original classification of an object.
# # This way I can simply check the classification by indexing to an ID number minus
# # the minimum ID number to find the classification. This prevented having an overly
# # large array but still has the speed advantage of indexing.
# minID = np.min(deepID)
# deepGalID = np.zeros(np.max(deepID) - minID + 1)
# deepGalID[deepID - minID] = deepClass

# GALA_GMAG = matGalaData['meas_psf_mag'][:,0]
# GALA_RMAG = matGalaData['meas_psf_mag'][:,1]

# GALA_CLASS = matGalaData['meas_EXTENDED_CLASS_SOF']

# GALA_GMAG_CM = matGalaData['meas_cm_mag'][:,0]
# GALA_RMAG_CM = matGalaData['meas_cm_mag'][:,1]

# GALA_GMAG[np.where(GALA_CLASS >= classCutoff)[0]] = GALA_GMAG_CM[np.where(GALA_CLASS >= classCutoff)[0]]
# GALA_RMAG[np.where(GALA_CLASS >= classCutoff)[0]] = GALA_RMAG_CM[np.where(GALA_CLASS >= classCutoff)[0]]

# GALA_RA = galaPositions['meas_ra']
# GALA_DEC = galaPositions['meas_dec']
# GALA_PIX = hp.ang2pix(res, GALA_RA, GALA_DEC, nest = True, lonlat = True)

# GALA_MAT_ID = matGalaID['bal_id']
# GALA_TRUE_ID = galaTrueID['true_id']
# GALA_MAT_SORT = GALA_MAT_ID.argsort()

# GALA_FLAG_ID = detGalaID['bal_id']

# FOREGROUND = galaCutFlags['flags_foreground']
# BADREGIONS = galaCutFlags['flags_badregions']
# FOOTPRINT = galaCutFlags['flags_footprint']
# ARCSECONDS = galaCutFlags['match_flag_1.5_asec']

# FLAG_SORT = GALA_FLAG_ID.argsort()
# GALA_FLAG_ID = GALA_FLAG_ID[FLAG_SORT]
# FOREGROUND = FOREGROUND[FLAG_SORT]
# BADREGIONS = BADREGIONS[FLAG_SORT]
# FOOTPRINT = FOOTPRINT[FLAG_SORT]
# ARCSECONDS = ARCSECONDS[FLAG_SORT]
    
# FLAG_CROP = np.isin(GALA_FLAG_ID, GALA_MAT_ID)

# FOREGROUND = FOREGROUND[FLAG_CROP]
# BADREGIONS = BADREGIONS[FLAG_CROP]
# FOOTPRINT = FOOTPRINT[FLAG_CROP]
# ARCSECONDS = ARCSECONDS[FLAG_CROP]

# FOREGROUND_PAR = np.zeros_like(FOREGROUND)
# BADREGIONS_PAR = np.zeros_like(FOREGROUND)
# FOOTPRINT_PAR = np.zeros_like(FOREGROUND)
# ARCSECONDS_PAR = np.zeros_like(FOREGROUND)

# FOREGROUND_PAR[GALA_MAT_SORT] = FOREGROUND
# BADREGIONS_PAR[GALA_MAT_SORT] = BADREGIONS
# FOOTPRINT_PAR[GALA_MAT_SORT] = FOOTPRINT
# ARCSECONDS_PAR[GALA_MAT_SORT] = ARCSECONDS

# GALA_MG = GALA_GMAG - mu
# GALA_GR = GALA_GMAG - GALA_RMAG

# filterSelection=Path.contains_points(path,np.vstack((GALA_GR,GALA_MG)).T)

# PIX_CUT = np.isin(GALA_PIX, validPix)

# galaCrop = np.where((FOREGROUND_PAR == 0) &
#                     (BADREGIONS_PAR < 2) & 
#                     (FOOTPRINT_PAR == 1) &
#                     (ARCSECONDS < 2) &
#                     (GALA_RMAG <= rMagCut) &
#                     (GALA_CLASS >= 0) &
#                     (GALA_CLASS <= 3) & 
#                     (PIX_CUT) & 
#                     (filterSelection) &
#                     (deepGalID[GALA_TRUE_ID - minID] == 1))[0]

In [19]:
# from astropy.table import Table

# gala_table = Table()
# for prop in objectProps:
#     gala_table[prop] = matGalaData[prop][galaCrop]
# for posProp in positionProps:
#     gala_table[posProp] = galaPositions[posProp][galaCrop]
# gala_table.write(galaDataFile, overwrite = True)

In [20]:
galaDataExt = fitsio.read(galaDataFile)

In [21]:
inputs = tf.keras.Input(shape=(parameterCount))

x = tf.keras.layers.Dense(300, activation="relu")(inputs)
x = tf.keras.layers.Dense(300, activation="relu")(x)
skip_connection = inputs
x = tf.keras.layers.concatenate([x, skip_connection])

x = tf.keras.layers.Dense(100, activation="relu")(x)
x = tf.keras.layers.Dense(100, activation="relu")(x)
x = tf.keras.layers.concatenate([x, skip_connection])

x = tf.keras.layers.Dense(30, activation="relu")(x)
x = tf.keras.layers.Dense(30, activation="relu")(x)
x = tf.keras.layers.concatenate([x, skip_connection])

outputs = tf.keras.layers.Dense(2, activation = 'softmax')(x)

2023-12-13 18:34:35.204408: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [22]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [23]:
# Two purposes of this: first, a speed up to not use every galaxy.
# Secondly, it will make star and galaxy counts more similar to make the network perform more evenly?
perGalasToUse = 0.6

In [24]:
len(starData) / (perGalasToUse*len(galaDataExt) + len(starData))

0.1977569426495906

In [25]:
useGalaInds = np.full(len(galaDataExt), False, dtype = bool)
useGalaInds[0:int(perGalasToUse*len(galaDataExt))] = True
np.random.shuffle(useGalaInds)

In [26]:
galaData = galaDataExt[useGalaInds]

In [27]:
trainGala, valGala, testGala = trainValTest(len(galaData), 0.8, 0.1)

In [28]:
trainStar, valStar, testStar = trainValTest(len(starData), 0.8, 0.1)

# NEXT STEPS:

Crop data, maybe do some shuffling to get a random distribution (not entirely sure how batches are picked by keras). Then add in survey property data based on positional data and run some training.

In [29]:
galaPix = hp.ang2pix(res, galaData[positionProps[0]], galaData[positionProps[1]], nest = True, lonlat = True)

In [30]:
starPix = hp.ang2pix(res, starData[positionProps[0]], starData[positionProps[1]], nest = True, lonlat = True)

In [31]:
galaSurveyProps = np.zeros((92, len(galaPix)), dtype = float)
starSurveyProps = np.zeros((92, len(starPix)), dtype = float)

origCondFiles = balrConfig.origCondFiles
stelFile = balrConfig.stelFile
    
for i in range(len(origCondFiles)):
    condData = fitsio.read(origCondFiles[i]) # This reads in the data.
    condSigExt = np.full(12*(4096**2), -1.6375e+30) # Gives a default value.
    condSigExt[condData['PIXEL']] = condData['SIGNAL'] # Changes all valid pixels to their corresponding signals.
    galaSurveyProps[i] = condSigExt[galaPix]
    starSurveyProps[i] = condSigExt[starPix]

stelDensExt = fitsio.read(stelFile)['I'].flatten() # Loads in the stellar density.
galaSurveyProps[-1] = stelDensExt[galaPix]
starSurveyProps[-1] = stelDensExt[starPix]

In [32]:
galaObjectProps = np.zeros((parameterCount - 92, len(galaPix)), dtype = float)
initial_index = 0
final_index = 0
while initial_index < len(objectProps):
    
    if type(galaData[objectProps[initial_index]][0]) == np.ndarray:
        for j in range(len(galaData[objectProps[initial_index]][0])):
            galaObjectProps[final_index] = galaData[objectProps[initial_index]][:,j]
            final_index += 1
    else:
        galaObjectProps[final_index] = galaData[objectProps[initial_index]]
        final_index += 1
        
    initial_index += 1

In [33]:
starObjectProps = np.zeros((parameterCount - 92, len(starPix)), dtype = float)
initial_index = 0
final_index = 0
while initial_index < len(objectProps):
    
    if type(starData[objectProps[initial_index]][0]) == np.ndarray:
        for j in range(len(starData[objectProps[initial_index]][0])):
            starObjectProps[final_index] = starData[objectProps[initial_index]][:,j]
            final_index += 1
    else:
        starObjectProps[final_index] = starData[objectProps[initial_index]]
        final_index += 1
        
    initial_index += 1

In [34]:
condMeans = np.zeros(92, dtype = float)
condStds = np.zeros(92, dtype = float)

objMeans = np.zeros(parameterCount - 92, dtype = float)
objStds = np.zeros(parameterCount - 92, dtype = float)

for i in range(len(condMeans)):
    condMeans[i] = np.average(np.append(galaSurveyProps[i], starSurveyProps[i]))
    condStds[i] = np.std(np.append(galaSurveyProps[i], starSurveyProps[i]))
    
for i in range(len(objMeans)):
    objMeans[i] = np.average(np.append(galaObjectProps[i], starObjectProps[i]))
    objStds[i] = np.std(np.append(galaObjectProps[i], starObjectProps[i]))

In [35]:
normedGalaData = np.zeros((parameterCount, len(galaPix)), dtype = float)
for i in range(parameterCount):
    if i < 92:
        normedGalaData[i] = (galaSurveyProps[i] - condMeans[i]) / condStds[i]
    else:
        normedGalaData[i] = (galaObjectProps[i-92] - objMeans[i-92]) / objStds[i-92]
normedGalaData = normedGalaData.T
        
normedStarData = np.zeros((parameterCount, len(starPix)), dtype = float)
for i in range(parameterCount):
    if i < 92:
        normedStarData[i] = (starSurveyProps[i] - condMeans[i]) / condStds[i]
    else:
        normedStarData[i] = (starObjectProps[i-92] - objMeans[i-92]) / objStds[i-92]
normedStarData = normedStarData.T

In [36]:
galaTrain = normedGalaData[trainGala]
galaVal = normedGalaData[valGala]
galaTest = normedGalaData[testGala]

starTrain = normedStarData[trainStar]
starVal = normedStarData[valStar]
starTest = normedStarData[testStar]

## Classification Scheme:

[1, 0] = Galaxy

[0, 1] = Star

In [37]:
newClassifiers = np.unique(starTest[:,-1])

In [38]:
XTrain = np.append(galaTrain, starTrain, axis = 0)

YTrain0 = np.full(len(XTrain), True, dtype = bool)
YTrain0[len(galaTrain):] = False

YTrain1 = ~YTrain0

YTrain = np.zeros((len(XTrain), 2))
YTrain[:,0] = YTrain0.astype(int)
YTrain[:,1] = YTrain1.astype(int)

trainShuffle = np.arange(len(YTrain))
np.random.shuffle(trainShuffle)

XTrain = XTrain[trainShuffle]
YTrain = YTrain[trainShuffle]

XVal = np.append(galaVal, starVal, axis = 0)

YVal0 = np.full(len(XVal), True, dtype = bool)
YVal0[len(galaVal):] = False

YVal1 = ~YVal0

YVal = np.zeros((len(XVal), 2))
YVal[:,0] = YVal0.astype(int)
YVal[:,1] = YVal1.astype(int)

In [39]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
initialStarAccuracy = len(np.where(starTest[:,-1] < np.average(newClassifiers))[0]) / len(starTest)
initialStarAccuracy

0.7099513341164625

In [41]:
initialGalaAccuracy = len(np.where(galaTest[:,-1] > np.average(newClassifiers))[0]) / len(galaTest)
initialGalaAccuracy

0.9300890227355462

In [ ]:
checkpointFile = r'SavedModels/60_Percent_of_Galaxies'

modelCheckpointCallback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpointFile,
    monitor='val_accuracy',
    verbose=1,
    mode='max',
    save_best_only=True,
    save_weights_only=True)

model.fit(XTrain, YTrain, epochs=30, batch_size=50, validation_data=(XVal, YVal), callbacks=[modelCheckpointCallback])
# model.fit(XTrain, YTrain, epochs=30, batch_size=100, validation_data=(XVal, YVal), callbacks=[modelCheckpointCallback])
# model.fit(XTrain, YTrain, epochs=30, batch_size=500, validation_data=(XVal, YVal), callbacks=[modelCheckpointCallback])
# model.fit(XTrain, YTrain, epochs=30, batch_size=2500, validation_data=(XVal, YVal), callbacks=[modelCheckpointCallback])

Epoch 1/30
24106/24106 [==============================] - ETA: 0s - loss: 0.2440 - accuracy: 0.9058
Epoch 1: val_accuracy improved from -inf to 0.91019, saving model to SavedModels/60_Percent_of_Galaxies
24106/24106 [==============================] - 558s 23ms/step - loss: 0.2440 - accuracy: 0.9058 - val_loss: 0.2331 - val_accuracy: 0.9102
Epoch 2/30
24105/24106 [============================>.] - ETA: 0s - loss: 0.2297 - accuracy: 0.9111
Epoch 2: val_accuracy improved from 0.91019 to 0.91156, saving model to SavedModels/60_Percent_of_Galaxies
24106/24106 [==============================] - 580s 24ms/step - loss: 0.2297 - accuracy: 0.9111 - val_loss: 0.2291 - val_accuracy: 0.9116
Epoch 3/30
24106/24106 [==============================] - ETA: 0s - loss: 0.2251 - accuracy: 0.9127
Epoch 3: val_accuracy improved from 0.91156 to 0.91243, saving model to SavedModels/60_Percent_of_Galaxies
24106/24106 [==============================] - 588s 24ms/step - loss: 0.2251 - accuracy: 0.9127 - val_loss

In [ ]:
YStarTest0 = np.zeros(len(starTest))
YStarTest1 = np.ones(len(starTest))
YStarTest = np.zeros((len(starTest), 2))
YStarTest[:,0] = YStarTest0
YStarTest[:,1] = YStarTest1

In [ ]:
starScore = model.evaluate(starTest, YStarTest, verbose=0)
print("Test loss:", format(starScore[0],".4f"))
print("Test accuracy:", format(starScore[1],".5f"))

In [ ]:
YGalaTest0 = np.ones(len(galaTest))
YGalaTest1 = np.zeros(len(galaTest))
YGalaTest = np.zeros((len(galaTest), 2))
YGalaTest[:,0] = YGalaTest0
YGalaTest[:,1] = YGalaTest1

In [ ]:
galaScore = model.evaluate(galaTest, YGalaTest, verbose=0)
print("Test loss:", format(galaScore[0],".4f"))
print("Test accuracy:", format(galaScore[1],".5f"))